In [7]:
import ollama
import sentencepiece
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [8]:
# 훈련 데이터셋 선언

data_train = pd.read_csv("../data/train.csv")
print(len(data_train))

2800


In [9]:
# 텍스트 오염 데이터셋 선언

df_contaminated_texts = pd.read_csv("../data/df_contaminated_texts.csv")
df_contaminated_texts = df_contaminated_texts.drop(["special_char_count", "special_char_ratio"], axis=1)
print(len(df_contaminated_texts))


#  텍스트 오염 데이터셋 타겟을 기존대로 숫자형으로 복구시켜줌

def mapping_label(df):
    # 딕셔너리로 매핑 정의
    label_map = {
        0:'생활문화',1:'스포츠',2:'정치',3:'사회',4:'IT과학',5:'경제',6:'세계'
    }
    inverted_label_map = {value: key for key, value in label_map.items()}

    # map 함수를 사용하여 변환
    df['target'] = df['target'].map(inverted_label_map)
    df['target'] = df['target'].map(lambda x: int(x))
    
    return df

# 사용
data_contaminated_texts = mapping_label(df_contaminated_texts)

1595


In [10]:
# 텍스트 비오염 데이터셋 선언

data_clean_texts = pd.DataFrame(columns=data_train.columns)
data_contaminated_texts_ids = data_contaminated_texts['ID'].tolist()

for i in range(len(data_train)):
    id = data_train.iloc[i]["ID"]
    if id not in data_contaminated_texts_ids:
        data_clean_texts.loc[len(data_clean_texts)] = data_train.iloc[i]

for i in range(10):
    print(data_clean_texts.iloc[i])

ID                  ynat-v1_train_00003
text      갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
target                                5
Name: 0, dtype: object
ID                   ynat-v1_train_00005
text      美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다
target                                 0
Name: 1, dtype: object
ID                     ynat-v1_train_00007
text      아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...
target                                   4
Name: 2, dtype: object
ID                   ynat-v1_train_00008
text      朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합
target                                 6
Name: 3, dtype: object
ID               ynat-v1_train_00009
text      듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감
target                             4
Name: 4, dtype: object
ID         ynat-v1_train_00011
text      NH투자 1월 옵션 만기일 매도 우세
target                       1
Name: 5, dtype: object
ID             ynat-v1_train_00012
text      황총리 각 부처 비상대비태세 철저히 강구해야
target                           2
Name: 6, dtype: object
ID                   ynat-v1_train_

In [11]:
titles_text_clean = []

for i in range(len(data_clean_texts)):
    titles_text_clean.append(data_clean_texts["text"])


In [12]:
titles_text_contaminated = [[] for i in range(7)]

for i in range(len(df_contaminated_texts)):
    target = df_contaminated_texts.iloc[i]["target"]
    titles_text_contaminated[target].append(df_contaminated_texts.iloc[i]["text"])

In [ ]:
# Models sorted by task: text generation, language: Korean, downloads 및 trending 각각 상위 3개

model_names = ["OrionStarAI/Orion-14B-Base", 
               "MLP-KTLim/llama-3-Korean-Bllossom-8B",
               "beomi/polyglot-ko-12.8b-safetensors",
               "CohereForAI/aya-expanse-8b",
               "CohereForAI/aya-expanse-32b",
               "Bllossom/llama-3.2-Korean-Bllossom-3B"]

# 입력 텍스트
count = 0
count_limit = 25
input_text = ""
for title in titles_text_clean[0]:
    input_text += f"{title}[break]" 
    count += 1
    if count >= count_limit:
        break

print(input_text)

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

    prompt = f"""
    다음은 여러 뉴스 기사 제목입니다. 각 제목은 [break]로 구분되어 있습니다.
    다음 뉴스 기사 제목들은 총 7가지의 주제로 나누어져 있는데. 그 7가지 주제가 무엇인지 맞춰주세요.

    뉴스 제목:
    {input_text}

    """

    # 입력 텍스트를 토큰화
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # 모델로부터 텍스트 생성
    max_length = tokenizer.model_max_length
    print("모델의 최대 길이:", max_length)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids,
            max_length=300,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=2,
            eos_token_id=tokenizer.eos_token_id  # 종료 토큰 설정
        )


    # 생성된 토큰을 텍스트로 변환
    output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # 조건문을 사용하여 출력할 텍스트 결정
    if len(output_text) > len(input_text):
        # output_text가 input_text보다 길다면 input_text 부분을 잘라낸 후 출력
        response_text = output_text[len(input_text):].strip()
    else:
        # output_text가 input_text보다 짧거나 같다면 그대로 출력
        response_text = output_text.strip()

    print("모델 응답:", response_text)

갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩[break]美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다[break]아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...[break]朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합[break]듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감[break]NH투자 1월 옵션 만기일 매도 우세[break]황총리 각 부처 비상대비태세 철저히 강구해야[break]게시판 KISA 박민정 책임연구원 APTLD 이사 선출[break]공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예[break]월세 전환에 늘어나는 주거비 부담…작년 역대 최고치[break]페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공[break]추신수 타율 0.265로 시즌 마감…최지만은 19홈런·6...[break]아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...[break]서울에 다시 오존주의보…도심·서북·동북권 발령종합[break]안보리 대북결의안 2270호 이행보고서 제출한 나라 70개 육박[break]게시판 KBS 코로나가 바꾼 일상 대국민 영상 공모[break]크루즈 관광객용 반나절 부산 해안 트레킹 상품 개발[break]마틴 우리카드 코치로 V리그 복귀…한국 좋아요[break]LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택[break]피폭 노동자가 증언하는 후쿠시마 원전 은폐와 속임수[break]한국민족의 뿌리가 북방…5천년 전 한반도서 자생적으로 형성[break]근육조직 생성에 필수적인 열쇠 발견[break]유엔 리비아 내전 악화·국제적 확산 우려…아랍연맹 긴급회의종합[break]공룡 파충류보다는 타조·매 등 새에 가깝다[break]대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합[break]영상 선출부터 퇴장까지…나경원 원내대표 파란만장 1년[break]MBC 류현진 특집 다큐…MLB 전설에게 듣는 류현진[break]모바일 컴퓨팅 미래는…서울서 국제학회 ACM 모비시스 개막[br

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


모델의 최대 길이: 4096


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

In [ ]:
# Models sorted by task: text generation, language: Korean, downloads 및 trending 각각 상위 3개

model_names = ["OrionStarAI/Orion-14B-Base", 
               "MLP-KTLim/llama-3-Korean-Bllossom-8B",
               "beomi/polyglot-ko-12.8b-safetensors",
               "CohereForAI/aya-expanse-8b",
               "CohereForAI/aya-expanse-32b",
               "Bllossom/llama-3.2-Korean-Bllossom-3B"]

# 입력 텍스트
count = 0
count_limit = 25
input_text = ""
for title in titles_text_clean[0]:
    input_text += f"{title}[break]" 
    count += 1
    if count >= count_limit:
        break

print(input_text)

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

    prompt = f"""
    다음은 여러 뉴스 기사 제목입니다. 각 제목은 [break]로 구분되어 있습니다.
    다음 뉴스 기사 제목들은 총 7가지의 주제로 나누어져 있는데. 그 7가지 주제가 무엇인지 맞춰주세요.

    뉴스 제목:
    {input_text}

    """

    # 입력 텍스트를 토큰화
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # 모델로부터 텍스트 생성
    max_length = tokenizer.model_max_length
    print("모델의 최대 길이:", max_length)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids,
            max_length=max_length,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=2,
            eos_token_id=tokenizer.eos_token_id  # 종료 토큰 설정
        )


    # 생성된 토큰을 텍스트로 변환
    output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # 조건문을 사용하여 출력할 텍스트 결정
    if len(output_text) > len(input_text):
        # output_text가 input_text보다 길다면 input_text 부분을 잘라낸 후 출력
        response_text = output_text[len(input_text):].strip()
    else:
        # output_text가 input_text보다 짧거나 같다면 그대로 출력
        response_text = output_text.strip()

    print("모델 응답:", response_text)

    import gc
    del model
    del tokenizer
    gc.collect()

갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩[break]美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다[break]아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...[break]朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합[break]듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감[break]NH투자 1월 옵션 만기일 매도 우세[break]황총리 각 부처 비상대비태세 철저히 강구해야[break]게시판 KISA 박민정 책임연구원 APTLD 이사 선출[break]공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예[break]월세 전환에 늘어나는 주거비 부담…작년 역대 최고치[break]페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공[break]추신수 타율 0.265로 시즌 마감…최지만은 19홈런·6...[break]아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...[break]서울에 다시 오존주의보…도심·서북·동북권 발령종합[break]안보리 대북결의안 2270호 이행보고서 제출한 나라 70개 육박[break]게시판 KBS 코로나가 바꾼 일상 대국민 영상 공모[break]크루즈 관광객용 반나절 부산 해안 트레킹 상품 개발[break]마틴 우리카드 코치로 V리그 복귀…한국 좋아요[break]LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택[break]피폭 노동자가 증언하는 후쿠시마 원전 은폐와 속임수[break]한국민족의 뿌리가 북방…5천년 전 한반도서 자생적으로 형성[break]근육조직 생성에 필수적인 열쇠 발견[break]유엔 리비아 내전 악화·국제적 확산 우려…아랍연맹 긴급회의종합[break]공룡 파충류보다는 타조·매 등 새에 가깝다[break]대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합[break]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


모델의 최대 길이: 4096


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


ValueError: Input length of input_ids is 527, but `max_length` is set to 300. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.